In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import xlwings as xw

def load_KB_Data(path):
    wb = xw.Book(path)
    sheet = wb.sheets['2.매매APT']
    row_num = sheet.range((1,1)).end('down').end('down').end('down').row
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value
    
    return raw_data
    
path = r'data/(월간)KB주택가격동향_시계열(20211126).xlsx'
data = load_KB_Data(path)
data

,구분,전국,서울,강북,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,양산,거제,진주,김해,통영,제주도,제주/\n서귀포,기타지방
0,None,None,None,None,강북구,광진구,노원구,도봉구,동대문구,마포구,...,의창구,진해구,None,None,None,None,None,None,None,None
1,Classification,Total,Seoul,Northern seoul,Gangbuk-gu,Gwangjin-gu,Nowon-gu,Dobong-gu,Dongdaemun-gu,Mapo-gu,...,Uichang,Jinhae,Yangsan,Geoje,Jinju,Kimhae,Tongyoung,Jeju-do,Jeju/\nSeogwipo,Non-Metropolitan Area
2,86.1,20.973143,17.152933,24.274004,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,2.0,20.973143,17.182661,24.234211,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,3.0,20.934518,17.152933,24.194418,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,6.0,120.246305,126.177277,127.957798,128.044911,123.201071,140.097495,129.969038,122.803156,129.185363,...,108.676149,106.714819,107.291705,100.088407,99.325184,105.370034,100.946001,None,103.499302,106.076314
428,7.0,122.073099,127.795184,129.674244,129.940662,125.065092,142.414689,132.009702,124.084161,130.76635,...,109.083161,106.832476,108.13741,100.716276,100.200671,106.149786,101.242069,None,105.794801,107.059163
429,8.0,124.495019,129.831503,131.920188,131.739377,126.99974,145.26873,135.090078,126.447974,132.642771,...,109.289778,107.867171,109.874122,101.728765,100.592694,108.424654,101.820351,None,109.394653,108.426427
430,9.0,126.945035,132.029481,134.387012,134.120969,129.111093,148.814433,137.037798,128.477923,134.057679,...,109.742781,108.634969,111.372418,103.413152,101.848495,110.484822,102.473656,None,111.807059,110.062803


In [2]:
def KBPriceIndex(data):

    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(data.columns)
    small_col = list(data.iloc[0])

    # 시-구 None 부분을 시-구로 변경
    for num, gu_data in enumerate(small_col):
        if gu_data is None:
            small_col[num] = big_col[num]
            
        check = num
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break;
            else:
                check = check -1

    big_col[129] = '경기'
    big_col[130] = '경기'
    big_col[185] = '서귀포'
    
    data.columns = [big_col, small_col]
    new_col_data = data.drop([0,1])
    
    index_list = list(new_col_data['구분']['구분'])

    new_index = []

    for num, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        if int(temp[0]) > 12:
            if len(temp[0]) == 2:
                new_index.append('19' + temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
            new_index.append(new_index[num-1].split('.')[0] + '.' + temp[0])

    # 만들어진 날짜 리스트를 인덱스로 설정 
    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data  = new_col_data.drop(('구분', '구분'), axis=1)
    
    return cleaned_data
    
kb_data = KBPriceIndex(data)
kb_data

전국          서울                                      \
                    전국          서울          강북         강북구         광진구   
1986-01-01   20.973143   17.152933   24.274004        None        None   
1986-02-01   20.973143   17.182661   24.234211        None        None   
1986-03-01   20.934518   17.152933   24.194418        None        None   
1986-04-01   20.818644   17.034022   24.035244        None        None   
1986-05-01   20.664146   16.944839   23.716896        None        None   
...                ...         ...         ...         ...         ...   
2021-06-01  120.246305  126.177277  127.957798  128.044911  123.201071   
2021-07-01  122.073099  127.795184  129.674244  129.940662  125.065092   
2021-08-01  124.495019  129.831503  131.920188  131.739377   126.99974   
2021-09-01  126.945035  132.029481  134.387012  134.120969  129.111093   
2021-10-01  128.610661  133.410964  135.635701  135.625977  129.399253   

                                                                        ...  \
                   노원구         도봉구        동대문구         마포구        서대문구  ...   
1986-01-01        None        None        None        None        None  ...   
1986-02-01        None        None        None        None        None  ...   
1986-03-01        None        None        None        None        None  ...   
1986-04-01        None        None        None        None        None  ...   
1986-05-01        None        None        None        None        None  ...   
...                ...         ...         ...         ...         ...  ...   
2021-06-01  140.097495  129.969038  122.803156  129.185363  121.778978  ...   
2021-07-01  142.414689  132.009702  124.084161   130.76635  123.400345  ...   
2021-08-01   145.26873  135.090078  126.447974  132.642771  124.833203  ...   
2021-09-01  148.814433  137.037798  128.477923  134.057679  126.925999  ...   
2021-10-01  150.255739  138.426907  129.660525  135.059091  127.688398  ...   

                    경남                                                  \
                   의창구         진해구          양산          거제          진주   
1986-01-01        None        None        None        None        None   
1986-02-01        None        None        None        None        None   
1986-03-01        None        None        None        None        None   
1986-04-01        None        None        None        None        None   
1986-05-01        None        None        None        None        None   
...                ...         ...         ...         ...         ...   
2021-06-01  108.676149  106.714819  107.291705  100.088407   99.325184   
2021-07-01  109.083161  106.832476   108.13741  100.716276  100.200671   
2021-08-01  109.289778  107.867171  109.874122  101.728765  100.592694   
2021-09-01  109.742781  108.634969  111.372418  103.413152  101.848495   
2021-10-01  110.561422  109.470492  112.001544  103.839905  102.265991   

                                     제주도         서귀포        기타지방  
                    김해          통영   제주도    제주/\n서귀포        기타지방  
1986-01-01        None        None  None        None        None  
1986-02-01        None        None  None        None        None  
1986-03-01        None        None  None        None        None  
1986-04-01        None        None  None        None        None  
1986-05-01        None        None  None        None        None  
...                ...         ...   ...         ...         ...  
2021-06-01  105.370034  100.946001  None  103.499302  106.076314  
2021-07-01  106.149786  101.242069  None  105.794801  107.059163  
2021-08-01  108.424654  101.820351  None  109.394653  108.426427  
2021-09-01  110.484822  102.473656  None  111.807059  110.062803  
2021-10-01  111.917196   102.60193  None  114.243799  111.225797  

[430 rows x 186 columns]